In [83]:
pip install gigachain_community

Note: you may need to restart the kernel to use updated packages.


In [84]:
from langchain_community.chat_models.gigachat import GigaChat

In [85]:
credentials="NmZjNjg0NjktZDI2ZS00M2FmLTk0NmMtMTdmODljNjM4ZWZmOjBiZmQ1ZDI2LTczODUtNDQxYi04OWQyLWJlNTUzN2QzZWI4NQ=="

In [86]:
def load_user_prompt(file_path): # функция загрузки user_prompt из txt файла
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read().strip()

In [87]:
SYSTEM_PROMPT = """
Роль:
Ты — высококвалифицированный геймдизайнер и сценарист с многолетним опытом создания нарративных RPG. Твои работы известны глубокой проработкой сюжетных ветвлений, атмосферностью и интерактивностью.
Правила:
1. Строго следуй заданию – не отвечай на вопросы, не связанные с созданием квеста.
2. Избегай ненормативной лексики – если она встречается во входных данных, заменяй на нейтральные формулировки.
3. Соблюдай формат – вывод только в JSON, без пояснений, комментариев или отклонений от структуры.
Задание:
На вход подаются три параметра:
* Жанр (например, "киберпанк"),
* Главный герой (например, "хакер-одиночка"),
* Цель (например, "взломать замок на двери, найти и забрать чип с вирусом").
Требуется создать:
1. Квест в жанре RPG (5–10 сцен) в формате JSON.
2. Обязательные элементы каждой сцены:
   * scene_id – уникальный ID для ссылок через next_scene.
   * text – описание ситуации, окружения, диалогов или действий.
   * choices – список вариантов выбора игрока (минимум 2). Каждый выбор содержит:
      * text – описание действия,
      * next_scene : ID следующей сцены.
3. Обязательные условия:
   * Минимум одна развилка с последствиями (разные ветки сюжета).
   * Глубина одной альтернативной ветки – не менее 3 сцен (основная ветвь не считается).
   * В каждой сцене минимум 2 выбора (кроме финальных сцен).
Дополнительные требования к качеству:
* Атмосфера: Описания должны передавать настроение жанра (например, для киберпанка – неоновые улицы, корпоративный контроль, техно-детали).
* Логика: Выборы игрока должны влиять на сюжет (например, агрессивный подход приводит к бою, скрытный – к обходу препятствий).
* Вариативность: Хотя бы одна сцена должна иметь неочевидный выбор (например, довериться NPC или украсть его данные).
* * Основные требования к сюжету:
* История должна быть структурирована по схеме: завязка -> конфликт -> решение проблемы -> развязка.
* Персонажи (включая NPC) должны реагировать на выбор игрока, влияя на сюжет.
* В мире должны присутствовать:
   * Спутники (наёмные союзники, временные попутчики).
   * Ключевые персонажи (важные для основного сюжета).
   * Фоновые NPC (торговцы, стражники, обыватели).
   * Закадровые персонажи (легендарные герои, исторические фигуры, упомянутые в книгах/диалогах).
Диалоги:
* Смысл и уместность: Персонажи говорят только то, что соответствует их роли и ситуации.
   * Пример: Крестьянин не будет рассуждать о магических рунах, если он не связан с арканами.
   * Пример: Если город атакуют, диалоги должны отражать панику/срочность, а не бытовые темы.
* Характер речи зависит от:
   * Пола, возраста, статуса, образования и профессии персонажа.
      * Пример: Старый маг говорит мудро и архаично, юный вор — сленгово и дерзко.
* Интерактивность: Реакция NPC на игрока должна зависеть от его репутации, класса и прошлых решений.
   * Пример: Рыцаря встречают с почётом, бродягу — с подозрением.
Запреты:
* Персонажи не должны:
   * Говорить неестественно (например, ребёнок с лексикой учёного).
   * Игнорировать экстренные события (например, обсуждать урожай во время битвы).
   * Быть статичными (их отношение к игроку может меняться).


Пример вывода (JSON):
{


  "scenes": [
    {
      "scene_id": "start",
      "text": "Вы – {герой}. {Описание начальной ситуации}. Вокруг шумят неоновые вывески, а цель – {цель}.",
      "choices": [
        {
          "text": "Попробовать взломать дверь напрямую.",
          "next_scene": "hacking_attempt"
        },
        {
          "text": "Найти обходной путь через вентиляцию.",
          "next_scene": "vent_route"
        }
      ]
    },
    {
      "scene_id": "hacking_attempt",
      "text": "Взлом запущен... но система защиты обнаружила вас!",
      "choices": [
        {
          "text": "Быстро отключиться и скрыться.",
          "next_scene": "escape"
        },
        {
          "text": "Попробовать подавить защиту вирусом.",
          "next_scene": "virus_upload"
        }
      ]
    }
    // ... остальные сцены (5-10)
  ]


Критерии успешного выполнения:
* Четкое следование JSON-формату.
* Глубина ветвления (одна альтернативная ветка из 3+ сцен).
* Соответствие жанру и логике персонажа.
* Интерактивность (выборы с последствиями).
* Соответствие правилам построения сюжета.
Важно: Не добавляй пояснений – только чистый JSON!
"""

In [88]:
from httpx import ReadTimeout

In [89]:
# функция для генерации квеста
# Загружаем user_prompt
def generate_rpg_quest(user_prompt_txt_path, credentials):
    user_prompt = load_user_prompt(user_prompt_txt_path)

    try:
        giga = GigaChat(
            credentials=credentials,
            verify_ssl_certs=False,
            timeout=360,
            model="GigaChat-2-Max"
        )

        full_prompt = f"{SYSTEM_PROMPT}\n\nВходные данные:\n{user_prompt}\n\nВывод только в JSON!"

        response = giga.invoke(full_prompt)

        response_text = response.content

        return response_text

    except ReadTimeout:
        print("Ошибка: превышено время ожидания ответа от GigaChat.")
        return {"error": "timeout"}
    except Exception as e:
        print(f"Критическая ошибка: {str(e)}")
        return {"error": str(e)}


In [90]:
import json

In [91]:
if __name__ == "__main__":
    # Путь к файлу с user_prompt (например, "input.txt")
    USER_PROMPT_PATH = "input/example-3.txt"  # Замените на ваш путь

    # Авторизационный ключ GigaChat
    CREDENTIALS = credentials  # Замените на реальный ключ

    # Генерация квеста
    quest = generate_rpg_quest(USER_PROMPT_PATH, CREDENTIALS)

    # Сохранение результата в файл
    with open("text_output/example-3.txt", "w", encoding="utf-8") as f:
        f.write(quest)

    print("Квест успешно сгенерирован и сохранён в text_output/example-3.txt!")

Квест успешно сгенерирован и сохранён в text_output/example-3.txt!


In [93]:
import subprocess
import os
from time import sleep

def process_quest_with_validation(quest_name, max_retries=3, credentials=None, user_prompt_path=None):
    """
    Обрабатывает квест через process.py, который:
    1. Читает text_output/{quest_name}.txt
    2. Конвертирует в JSON
    3. Сохраняет в generated_quests/{quest_name}.json
    4. Выполняет валидацию

    При ошибках в stderr перезапускает генерацию квеста (до max_retries раз)
    """
    retry_count = 0

    while retry_count < max_retries:
        try:
            # Проверяем существование входного файла
            input_file = f"text_output/{quest_name}.txt"
            if not os.path.exists(input_file):
                print(f"Ошибка: файл {input_file} не найден")
                return False

            print(f"Обрабатываем квест: {quest_name} (попытка {retry_count + 1}/{max_retries})")
            print(f"Входной файл: {input_file}")

            # Запускаем process.py
            result = subprocess.run([
                'python3', 'process.py', f"text_output/{quest_name}.txt", "-o", f"generated_quests/{quest_name}.json"
            ],
            capture_output=True,
            text=True
            )

            # Выводим результат
            if result.stdout:
                print("Вывод process.py:")
                print(result.stdout)

            if result.stderr:
                print("Ошибки:")
                print(result.stderr)

                # Если есть ошибки и мы можем перегенерировать квест
                if credentials is not None and user_prompt_path is not None:
                    retry_count += 1
                    if retry_count < max_retries:
                        print(f"\n⚠️ Обнаружены ошибки, перегенерируем квест (попытка {retry_count + 1}/{max_retries})")

                        # Перегенерируем квест
                        from your_generation_module import generate_rpg_quest  # Импортируем функцию генерации

                        quest = generate_rpg_quest(user_prompt_path, credentials)

                        # Сохраняем новый вариант
                        with open(input_file, "w", encoding="utf-8") as f:
                            f.write(quest)

                        print("Квест перегенерирован, повторяем обработку...")
                        sleep(1)
                        continue

            if result.returncode == 0:
                print(f"✅ Квест {quest_name} успешно обработан и валидирован!")

                # Проверяем, что файл создался
                output_file = f"generated_quests/{quest_name}.json"
                if os.path.exists(output_file):
                    print(f"✅ Файл {output_file} создан")
                else:
                    print(f"⚠️ Файл {output_file} не найден")

                return True
            else:
                print(f"❌ Ошибка при обработке квеста {quest_name}")
                return False

        except Exception as e:
            print(f"Критическая ошибка: {e}")
            return False

    print(f"❌ Достигнуто максимальное количество попыток ({max_retries})")
    return False

# Обрабатываем example-3
if __name__ == "__main__":
    quest_name = "example-3"
    max_retries = 3  # Максимальное количество попыток перегенерации
    credentials = "your_credentials_here"  # Ваши учетные данные GigaChat
    user_prompt_path = "input/example-3.txt"  # Путь к файлу с промптом

    print(f"Запускаем обработку квеста: {quest_name}")
    print("=" * 50)

    success = process_quest_with_validation(
        quest_name=quest_name,
        max_retries=max_retries,
        credentials=credentials,
        user_prompt_path=user_prompt_path
    )
    
    if success:
        print("\n🎉 Обработка завершена успешно!")
        print(f"Квест {quest_name} готов к использованию в generated_quests/{quest_name}.json")
    else:
        print("\n❌ Обработка завершилась с ошибками")
        print("Проверьте логи выше для диагностики проблем")


Запускаем обработку квеста: example-3
Обрабатываем квест: example-3
Входной файл: text_output/example-3.txt
Вывод process.py:
Результат валидации: Все проверки пройдены успешно
JSON файл сохранен: generated_quests/example-3.json

✅ Квест example-3 успешно обработан и валидирован!
✅ Файл generated_quests/example-3.json создан

🎉 Обработка завершена успешно!
Квест example-3 готов к использованию в generated_quests/example-3.json
